In [ ]:
import numpy as np
import torch
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
df = pd.read_csv("train_lemma.csv")
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,lemmas
0,0000997932d777bf,explanationwhy the edits made under my usernam...,0,0,0,0,0,0,"['explanationwhy', 'the', 'edit', 'make', 'und..."
1,000103f0d9cfb60f,d'aww! he matches this background colour i'm s...,0,0,0,0,0,0,"[""d'aww"", 'he', 'match', 'this', 'background',..."
2,000113f07ec002fd,"hey man, i'm really not trying to edit war. it...",0,0,0,0,0,0,"['hey', 'man', 'I', 'be', 'really', 'not', 'tr..."
3,0001b41b1c6bb37e,morei can't make any real suggestions on impro...,0,0,0,0,0,0,"['morei', 'can', 'not', 'make', 'any', 'real',..."
4,0001d958c54c6e35,"you, sir, are my hero. any chance you remember...",0,0,0,0,0,0,"['you', 'sir', 'be', 'my', 'hero', 'any', 'cha..."
...,...,...,...,...,...,...,...,...,...
137136,ffe987279560d7ff,"and for the second time of asking, when your v...",0,0,0,0,0,0,"['and', 'for', 'the', 'second', 'time', 'of', ..."
137137,ffea4adeee384e90,you should be ashamed of yourself that is a ho...,0,0,0,0,0,0,"['you', 'should', 'be', 'ashamed', 'of', 'your..."
137138,ffee36eab5c267c9,"spitzer umm, theres no actual article for pros...",0,0,0,0,0,0,"['spitzer', 'umm', 'there', 's', 'no', 'actual..."
137139,fff125370e4aaaf3,and it looks like it was actually you who put ...,0,0,0,0,0,0,"['and', 'it', 'look', 'like', 'it', 'be', 'act..."


In [ ]:
print(f"Dataset shape: {df.shape}")

Dataset shape: (137141, 9)


In [ ]:
label_cols = [col for col in df.columns if col not in ['id', 'comment_text', 'lemmas']]
print(f"Label columns: {label_cols}")

Label columns: ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [ ]:
print("\nLabel distribution:")
for col in label_cols:
    pos_count = df[col].sum()
    total_count = len(df)
    print(f"  {col}: {pos_count}/{total_count} ({pos_count/total_count:.3f})")


Label distribution:
  toxic: 14084/137141 (0.103)
  severe_toxic: 1420/137141 (0.010)
  obscene: 7828/137141 (0.057)
  threat: 450/137141 (0.003)
  insult: 7354/137141 (0.054)
  identity_hate: 1289/137141 (0.009)


In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
print(f"Train shape: {train_df.shape}, Validation shape: {val_df.shape}")

Train shape: (109712, 9), Validation shape: (27429, 9)


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "FacebookAI/roberta-base",
    num_labels=len(label_cols),
    problem_type="multi_label_classification",
    use_safetensors=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(batch):
    encoding = tokenizer(
        batch["lemmas"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

    labels = []
    for i in range(len(batch["lemmas"])):
        label_row = [float(batch[col][i]) for col in label_cols]  # cast here
        labels.append(label_row)

    encoding["labels"] = labels
    return encoding

In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/109712 [00:00<?, ? examples/s]

Map:   0%|          | 0/27429 [00:00<?, ? examples/s]

In [ ]:
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Convert to numpy
    if isinstance(logits, torch.Tensor):
        logits = logits.cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()

    # Get probabilities using sigmoid
    probs = torch.sigmoid(torch.tensor(logits)).numpy()

    # Use simple 0.5 threshold for all labels
    preds = (probs >= 0.4).astype(int)

    print(f"Predictions shape: {preds.shape}")
    print(f"Labels shape: {labels.shape}")
    print(f"Positive predictions per label: {np.sum(preds, axis=0)}")
    print(f"Positive labels per label: {np.sum(labels, axis=0)}")

    # Compute metrics
    f1 = f1_score(labels, preds, average="micro", zero_division=0)
    precision = precision_score(labels, preds, average="micro", zero_division=0)
    recall = recall_score(labels, preds, average="micro", zero_division=0)

    # Per-label F1 for debugging
    f1_per_label = f1_score(labels, preds, average=None, zero_division=0)
    print(f"Per-label F1: {dict(zip(label_cols, f1_per_label))}")

    try:
        auc = roc_auc_score(labels, probs, average="micro")
    except ValueError:
        auc = 0.0

    return {
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "auc": auc
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./temp_results",
    eval_strategy="epoch",
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",
    remove_unused_columns=True,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=8,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-2721217396.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
print("Starting training...")
trainer.train()

# Final evaluation
print("\nFinal evaluation:")
final_metrics = trainer.evaluate()
print(f"Final F1: {final_metrics['eval_f1']:.4f}")
print(f"Final Precision: {final_metrics['eval_precision']:.4f}")
print(f"Final Recall: {final_metrics['eval_recall']:.4f}")
print(f"Final AUC: {final_metrics['eval_auc']:.4f}")

Starting training...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Auc
1,0.061900,0.049387,0.767058,0.709196,0.835201,0.987706
2,0.044500,0.044820,0.780387,0.734081,0.832929,0.989126
3,0.038100,0.045308,0.788307,0.759837,0.818994,0.988816
4,0.033200,0.047281,0.784411,0.735801,0.839897,0.989259
5,0.028900,0.048624,0.785396,0.747265,0.827628,0.988799


Predictions shape: (27429, 6)
Labels shape: (27429, 6)
Positive predictions per label: [3133  327 1772  161 1997  385]
Positive labels per label: [2811.  304. 1607.   97. 1508.  275.]
Per-label F1: {'toxic': np.float64(0.7950874831763123), 'severe_toxic': np.float64(0.4659270998415214), 'obscene': np.float64(0.840485350695472), 'threat': np.float64(0.5116279069767442), 'insult': np.float64(0.7646219686162625), 'identity_hate': np.float64(0.5393939393939394)}


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


...

Predictions shape: (27429, 6)
Labels shape: (27429, 6)
Positive predictions per label: [3022  409 1811   95 1830  369]
Positive labels per label: [2811.  304. 1607.   97. 1508.  275.]
Per-label F1: {'toxic': np.float64(0.8129607406137493), 'severe_toxic': np.float64(0.5049088359046283), 'obscene': np.float64(0.8490345231129316), 'threat': np.float64(0.5520833333333334), 'insult': np.float64(0.7771120431396046), 'identity_hate': np.float64(0.5993788819875776)}


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Predictions shape: (27429, 6)
Labels shape: (27429, 6)
Positive predictions per label: [2983  394 1764  110 1743  318]
Positive labels per label: [2811.  304. 1607.   97. 1508.  275.]
Per-label F1: {'toxic': np.float64(0.8098032447359337), 'severe_toxic': np.float64(0.5100286532951289), 'obscene': np.float64(0.8495995253633937), 'threat': np.float64(0.5893719806763285), 'insult': np.float64(0.7806828668102123), 'identity_hate': np.float64(0.6003372681281619)}

Final evaluation:


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Predictions shape: (27429, 6)
Labels shape: (27429, 6)
Positive predictions per label: [2816  547 1695   96 1688  274]
Positive labels per label: [2811.  304. 1607.   97. 1508.  275.]
Per-label F1: {'toxic': np.float64(0.8167762573307269), 'severe_toxic': np.float64(0.5076380728554641), 'obscene': np.float64(0.8534221683827983), 'threat': np.float64(0.5284974093264249), 'insult': np.float64(0.7903629536921152), 'identity_hate': np.float64(0.6193078324225865)}
Final F1: 0.7883
Final Precision: 0.7598
Final Recall: 0.8190
Final AUC: 0.9888


In [ ]:
save_dir = "Roberta_bert"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('Roberta_bert/tokenizer_config.json',
 'Roberta_bert/special_tokens_map.json',
 'Roberta_bert/vocab.json',
 'Roberta_bert/merges.txt',
 'Roberta_bert/added_tokens.json',
 'Roberta_bert/tokenizer.json')

In [ ]:
!zip -r Roberta_bert.zip Roberta_bert

  adding: Roberta_bert/ (stored 0%)
  adding: Roberta_bert/tokenizer.json (deflated 82%)
  adding: Roberta_bert/model.safetensors (deflated 13%)
  adding: Roberta_bert/merges.txt (deflated 53%)
  adding: Roberta_bert/config.json (deflated 55%)
  adding: Roberta_bert/special_tokens_map.json (deflated 52%)
  adding: Roberta_bert/vocab.json (deflated 59%)
  adding: Roberta_bert/tokenizer_config.json (deflated 75%)


In [ ]:
from google.colab import files
files.download("Roberta_bert.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>